In [4]:
import torch
torch.cuda.get_device_name(0)

'Tesla T4'

In [4]:
# Select CUDA device index
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model_name_or_path = "openai/whisper-large-v2"
language = "Marathi"
language_abbr = "mr"
task = "transcribe"
dataset_name = "mozilla-foundation/common_voice_11_0"

In [37]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset(dataset_name, language_abbr, split="train+validation")
common_voice["test"] = load_dataset(dataset_name, language_abbr, split="test")

print(common_voice)

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2245it [00:00, 142888.55it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...

Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2237it [00:00, 100726.33it/s]


Dataset common_voice_11_0 downloaded and prepared to /home/sagemaker-user/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/mr/11.0.0/2c65b95d99ca879b1b1074ea197b65e0497848fd697fdb0582e0f6b75b6f4da0. Subsequent calls will reuse this data.


Found cached dataset common_voice_11_0 (/home/sagemaker-user/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/mr/11.0.0/2c65b95d99ca879b1b1074ea197b65e0497848fd697fdb0582e0f6b75b6f4da0)


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 3927
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 1816
    })
})


In [7]:
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"]
)

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 3927
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1816
    })
})


In [8]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)


In [9]:
print(common_voice["train"][0])


{'audio': {'path': '/home/sagemaker-user/.cache/huggingface/datasets/downloads/extracted/45355dc8208f28fc370743a61b369050a76c147d3f5af75c3f091f1656725b64/common_voice_mr_30585613.mp3', 'array': array([-7.73250586e-15, -1.34315905e-13, -1.40829778e-13, ...,
        4.88429314e-06,  3.52330612e-06,  1.60117417e-06]), 'sampling_rate': 48000}, 'sentence': 'आईचे आजारपण वाढत चालले, तसतशी मथीही नीट खातपीतनाशी झाली.'}


In [10]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [11]:
print(common_voice["train"][0])


{'audio': {'path': '/home/sagemaker-user/.cache/huggingface/datasets/downloads/extracted/45355dc8208f28fc370743a61b369050a76c147d3f5af75c3f091f1656725b64/common_voice_mr_30585613.mp3', 'array': array([-5.58273178e-14, -1.42828860e-13,  4.61396580e-13, ...,
       -7.70475605e-06, -1.76384767e-06,  4.35184714e-06]), 'sampling_rate': 16000}, 'sentence': 'आईचे आजारपण वाढत चालले, तसतशी मथीही नीट खातपीतनाशी झाली.'}


In [12]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [13]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)


Map (num_proc=2):   0%|          | 0/3927 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1816 [00:00<?, ? examples/s]

In [14]:
common_voice["train"]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 3927
})

In [38]:
common_voice.save_to_disk("marathi-common-voice-processed")

Saving the dataset (0/8 shards):   0%|          | 0/3927 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/1816 [00:00<?, ? examples/s]

In [39]:
!aws s3 cp --recursive "marathi-common-voice-processed" s3://sagemaker-us-east-1-348052051973/

/bin/bash: line 1: aws: command not found
